In [ ]:
import pmcpy
import xarray as xr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import copy
from matplotlib.ticker import MultipleLocator
import seaborn as sns
import matplotlib.pyplot as plt

## Loading results (static vs time-variant temperature)

In [ ]:
group = [['SO4','NO3','Cl','NH4','MSA','ARO1','ARO2','ALK1','OLE1','API1','API2','LIM1','LIM2','CO3','Na','Ca','OIN','OC'], ['BC']]

Info = []

for scenario_index in range(100):
    scenario_str = f"scenario_{scenario_index:04d}"

    columns = ['Temp','Temp_static', 'DryMass_BC', 'DryMass_BC_static','DryMass_Other', 'DryMass_Other_static', 'gIndex', 'gIndex_static', 'OIndex', 'OIndex_static']
    df = pd.DataFrame(columns=columns)

    for file_index in range(2, 26):
        p = f"variant/scenarios/{scenario_str}/out_init/urban_plume_0001_{file_index:08d}.nc"
        pmc = pmcpy.load_pmc(p)
        data = xr.open_dataset(p)

        

        Temp = float(data.temperature.values)
        DryMass_BC = pmc.get_aero_mass_conc(group[1]).sum()
        DryMass_Other = pmc.get_aero_mass_conc(group[0]).sum()
        oIndex = pmc.get_mixing_state_index(group_list=group, diversity=False)
        Index = pmc.get_mixing_state_index(drop_list=["H2O"], diversity=False)

        p = f"static/static_scenario/{scenario_str}/out_init/urban_plume_0001_{file_index:08d}.nc"
        pmc = pmcpy.load_pmc(p)
        data = xr.open_dataset(p)

        Temp_static = float(data.temperature.values)
        DryMass_BC_static = pmc.get_aero_mass_conc(group[1]).sum()
        DryMass_Other_static = pmc.get_aero_mass_conc(group[0]).sum()
        oIndex_static = pmc.get_mixing_state_index(group_list=group, diversity=False)
        Index_static = pmc.get_mixing_state_index(drop_list=["H2O"], diversity=False)

        new_row = [Temp, Temp_static, DryMass_BC, DryMass_BC_static, DryMass_Other, DryMass_Other_static, Index, Index_static, oIndex, oIndex_static]
        new_df = pd.DataFrame([new_row], columns=df.columns)
        df = pd.concat([df, new_df], ignore_index=True)

    df['Temp_diff'] = ( df['Temp'] - df['Temp_static'])
    df['OIndex_diff'] = ( df['OIndex_static'] - df['OIndex'])
    df['gIndex_diff'] = ( df['gIndex_static'] - df['gIndex'])

    average_tempdiff = np.mean(df['Temp_diff'])
    average_Odiff = np.mean(df['OIndex_diff'])
    average_gdiff = np.mean(df['gIndex_diff'])
    average_gRatio = (average_gdiff / average_tempdiff)
    average_ORatio = (average_Odiff / average_tempdiff)

    longitude = float(data.longitude.values)
    latitude = float(data.latitude.values)


    print(f"Done {scenario_index}")

    Info.append([df, longitude, latitude, average_gRatio, average_ORatio])

## KDE plotting

In [ ]:
errors = np.array([])

for i in range(100):
    df = copy.deepcopy(Info[i][0])
    df.index = df.index + 1
    list = df['gIndex_diff'].values
    errors = np.concatenate((errors, list))


In [ ]:
filtered_errors = errors[errors < -0.005]


custom_palette = sns.color_palette(["orange"])


sns.set_palette(custom_palette)


sns.set_style("whitegrid")  


fig, ax = plt.subplots(figsize=(10, 3))

sns.kdeplot(data=filtered_errors, shade=True, label = "χ error (static run- time-variant run)")


plt.xlabel("Error Value", color='white')
plt.ylabel("Density", color='white')
plt.title("KDE of General Mixing State Index χ Errors (Underestimations)", color='white')
plt.legend(loc = "upper left")

# Set the background color of both the plot and the figure
fig.patch.set_facecolor('grey')  # Set the figure background color
ax.tick_params(colors='white')


# Remove the box around the plot
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['bottom'].set_visible(False)
ax.spines['left'].set_visible(False)


ax.set_facecolor('grey')  


plt.show()

In [ ]:
filtered_errors = errors[errors > 0.005]


custom_palette = sns.color_palette(["orange"])


sns.set_palette(custom_palette)


sns.set_style("whitegrid")  # Show gridlines


fig, ax = plt.subplots(figsize=(10, 3))

sns.kdeplot(data=filtered_errors, shade=True, label = "χ error (static run- time-variant run)")


plt.xlabel("Error Value", color='white')
plt.ylabel("Density", color='white')
plt.title("KDE of General Mixing State Index χ Errors (Overestimations)", color='white')
plt.legend()


fig.patch.set_facecolor('grey')  
ax.tick_params(colors='white')


# Remove the box around the plot
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['bottom'].set_visible(False)
ax.spines['left'].set_visible(False)


ax.set_facecolor('grey')  


plt.show()

In [ ]:
errors = np.array([])

for i in range(100):
    df = copy.deepcopy(Info[i][0])
    df.index = df.index + 1
    list = df['OIndex_diff'].values
    errors = np.concatenate((errors, list))

In [ ]:
filtered_errors = errors[errors < -0.05]


custom_palette = sns.color_palette(["orange"])


sns.set_palette(custom_palette)


sns.set_style("whitegrid") 


fig, ax = plt.subplots(figsize=(10, 3))

sns.kdeplot(data=filtered_errors, shade=True, label = "χₒ error (static run- time-variant run)")


plt.xlabel("Error Value", color='white')
plt.ylabel("Density", color='white')
plt.title("KDE of optical Mixing State Index χₒ Errors (Underestimations)", color='white')
plt.legend(loc = "upper left")


fig.patch.set_facecolor('grey')  # Set the figure background color
ax.tick_params(colors='white')



ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['bottom'].set_visible(False)
ax.spines['left'].set_visible(False)


ax.set_facecolor('grey')  


plt.show()

In [ ]:
filtered_errors = errors[errors > 0.05]


custom_palette = sns.color_palette(["orange"])


sns.set_palette(custom_palette)


sns.set_style("whitegrid") 


fig, ax = plt.subplots(figsize=(10, 3))

sns.kdeplot(data=filtered_errors, shade=True, label = "χₒ error (static run- time-variant run)")


plt.xlabel("Error Value", color='white')
plt.ylabel("Density", color='white')
plt.title("KDE of Optical Mixing State Index χₒ Errors (Overestimations)", color='white')
plt.legend()


fig.patch.set_facecolor('grey') 
ax.tick_params(colors='white')



ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['bottom'].set_visible(False)
ax.spines['left'].set_visible(False)


ax.set_facecolor('grey')  



plt.show()

Individual column analysis

In [ ]:
all = []

for i in range(100):
    df = copy.deepcopy(Info[i][0])
    df.index = df.index + 1
    ochi = df['gIndex_diff'].abs().max()
    all.append(ochi)

np.mean(all)

In [ ]:
coldest = None
coldID = None

for i in range(100):
    df = copy.deepcopy(Info[i][0])
    df.index = df.index + 1
    if (coldest == None):
        coldest = df['gIndex_diff'].abs().max()
        coldID = i
    elif df['gIndex_diff'].abs().max() > coldest:
        coldest = df['gIndex_diff'].abs().max()
        coldID = i

Single case analysis

In [ ]:
value = Info[95]
print(f"{value[1]}, {value[2]}")
df = copy.deepcopy(value[0])
df.index = df.index + 1

In [ ]:
# Create a figure and the left y-axis
fig, ax1 = plt.subplots(figsize=(10, 6))

# Plot DryMass_Other on the left y-axis
color = 'green'
ax1.set_xlabel('Hours past')
ax1.set_ylabel('Mixing State Index χₒ', color='black')
line1 = ax1.plot(df.index, df['OIndex'], marker='o', linestyle='-', color=color, label='Mixing state index χₒ (time-variant temperature)')
line2 = ax1.plot(df.index, df['OIndex_static'], marker='x', linestyle='-', color='r', label='Mixing state index χₒ (static temperature)')
ax1.tick_params(axis='y', labelcolor='black')

# Create a second y-axis on the right side for 'Temp'
ax2 = ax1.twinx()

# Set the x-axis tick locator to show every 3 hours
x_locator = MultipleLocator(base=3)
ax1.xaxis.set_major_locator(x_locator)

# Plot Temp on the right y-axis
color = 'blue'
ax2.set_ylabel('Temperature (K)', color=color)
line3 = ax2.plot(df.index, df['Temp'], linestyle='-', color=color, label='Temperature')
ax2.tick_params(axis='y', labelcolor=color)

# Combine the legends from both axes
lines = line1 + line2 + line3
labels = [line.get_label() for line in lines]
ax1.legend(lines, labels, loc='upper right')

# Title
plt.title('Evolution of Optical Property Mixing State Index χₒ')
fig.tight_layout()

# Show the plot
plt.show()


In [ ]:
# Create a figure and the left y-axis
fig, ax1 = plt.subplots(figsize=(10, 6))

# Plot DryMass_Other on the left y-axis
color = 'green'
ax1.set_xlabel('Hours past')
ax1.set_ylabel('Mixing State Index χ', color='black')
line1 = ax1.plot(df.index, df['gIndex'], marker='o', linestyle='-', color=color, label='Mixing state index χ (time-variant temperature)')
line2 = ax1.plot(df.index, df['gIndex_static'], marker='x', linestyle='-', color='r', label='Mixing state index χ (static temperature)')
ax1.tick_params(axis='y', labelcolor=color)

# Create a second y-axis on the right side for 'Temp'
ax2 = ax1.twinx()

# Set the x-axis tick locator to show every 3 hours
x_locator = MultipleLocator(base=3)
ax1.xaxis.set_major_locator(x_locator)


# Plot Temp on the right y-axis
color = 'blue'
ax2.set_ylabel('Temperature (K)', color=color)
line3 = ax2.plot(df.index, df['Temp'], linestyle='-', color=color, label='Temperature')
ax2.tick_params(axis='y', labelcolor=color)

# Combine the legends from both axes
lines = line1 + line2 + line3
labels = [line.get_label() for line in lines]
ax1.legend(lines, labels, loc='upper right')

# Title
plt.title('Evolution of General Mixing State Index χ')
fig.tight_layout()

# Show the plot
plt.show()


In [ ]:
# Create a figure and the left y-axis
fig, ax1 = plt.subplots(figsize=(10, 6))

# Plot DryMass_Other on the left y-axis
color = 'green'
ax1.set_xlabel('Hours past')
ax1.set_ylabel('BC Mass Concentration (kg m⁻³)', color='black')
line1 = ax1.plot(df.index, df['DryMass_BC'], marker='s', linestyle='-', color='blue', label='BC (time-variant temperature)')
line2 = ax1.plot(df.index, df['DryMass_BC_static'], marker='s', linestyle='-', color='purple', label='BC (static temperature)')
ax1.tick_params(axis='y', labelcolor='black')
ax1.yaxis.label.set_size(14)

# Create a second y-axis on the right side for 'Temp'
ax2 = ax1.twinx()

# Set the x-axis tick locator to show every 3 hours
x_locator = MultipleLocator(base=3)
ax1.xaxis.set_major_locator(x_locator)

# Plot Temp on the right y-axis
color = 'blue'
ax2.set_ylabel('Other Species Mass Concentration (kg m⁻³)', color='black')
line3 = ax2.plot(df.index, df['DryMass_Other'], marker='*', linestyle='-', color='red', label='Other Species (time-variant temperature)')
line4 = ax2.plot(df.index, df['DryMass_Other_static'], marker='*', linestyle='-', color='orange', label='Other Species (static temperature)')
ax2.tick_params(axis='y', labelcolor='black')
ax2.yaxis.label.set_size(14)

# Combine the legends from both axes
lines = line1 + line2 + line3 + line4
labels = [line.get_label() for line in lines]
ax1.legend(lines, labels, loc='upper left')

# Title
plt.title('Aerosol Mass Concentration')
fig.tight_layout()

# Show the plot
plt.show()

In [ ]:

df = None
columns = ['Temp','Temp_static', 'DryMass_BC', 'DryMass_BC_static','DryMass_Other', 'DryMass_Other_static', 'gIndex', 'gIndex_static', 'OIndex', 'OIndex_static']
df = pd.DataFrame(columns=columns)

for file_index in range(2, 26):
    p = f"static/static_scenario/scenario_0030/out_init/urban_plume_0001_{file_index:08d}.nc"
    pmc = pmcpy.load_pmc(p)
    data = xr.open_dataset(p)

    

    Temp = float(data.temperature.values)
    DryMass_BC = pmc.get_aero_mass_conc(group[1]).sum()
    DryMass_Other = pmc.get_aero_mass_conc(group[0]).sum()
    oIndex = pmc.get_mixing_state_index(group_list=group, diversity=False)
    Index = pmc.get_mixing_state_index(drop_list=["H2O"], diversity=False)

    p = f"static2/scenario_0030/out_init/urban_plume_0001_{file_index:08d}.nc"
    pmc = pmcpy.load_pmc(p)
    data = xr.open_dataset(p)

    Temp_static = float(data.temperature.values)
    DryMass_BC_static = pmc.get_aero_mass_conc(group[1]).sum()
    DryMass_Other_static = pmc.get_aero_mass_conc(group[0]).sum()
    oIndex_static = pmc.get_mixing_state_index(group_list=group, diversity=False)
    Index_static = pmc.get_mixing_state_index(drop_list=["H2O"], diversity=False)

    new_row = [Temp, Temp_static, DryMass_BC, DryMass_BC_static, DryMass_Other, DryMass_Other_static, Index, Index_static, oIndex, oIndex_static]
    new_df = pd.DataFrame([new_row], columns=df.columns)
    df = pd.concat([df, new_df], ignore_index=True)

df['Temp_diff'] = ( df['Temp_static'] - df['Temp'])
df['OIndex_diff'] = ( df['OIndex_static'] - df['OIndex'])
df['gIndex_diff'] = ( df['gIndex_static'] - df['gIndex'])
df['OIndex_ratio'] =  ( df['OIndex_diff'] / df['Temp_diff'])
df['gIndex_ratio'] =  ( df['gIndex_diff'] / df['Temp_diff'])

average_gRatio = df['gIndex_ratio'].mean()
average_ORatio = df['OIndex_ratio'].mean()

print(average_gRatio)
print(average_ORatio)
longitude = float(data.longitude.values)
latitude = float(data.latitude.values)

df.index = df.index + 1

print("Done")


In [ ]:

fig, ax1 = plt.subplots(figsize=(10, 6))


color = 'green'
ax1.set_xlabel('Hours past')
ax1.set_ylabel('BC Mass Concentration (kg m⁻³)', color='black')
line1 = ax1.plot(df.index, df['DryMass_BC'], marker='s', linestyle='-', color='blue', label='BC (static temperature run1)')
line2 = ax1.plot(df.index, df['DryMass_BC_static'], marker='s', linestyle='-', color='purple', label='BC (static temperature run2)')
ax1.tick_params(axis='y', labelcolor='black')
ax1.yaxis.label.set_size(14)


ax2 = ax1.twinx()


x_locator = MultipleLocator(base=3)
ax1.xaxis.set_major_locator(x_locator)


color = 'blue'
ax2.set_ylabel('Other Species Mass Concentration (kg m⁻³)', color='black')
line3 = ax2.plot(df.index, df['DryMass_Other'], marker='*', linestyle='-', color='red', label='Other Species (static temperature run1)')
line4 = ax2.plot(df.index, df['DryMass_Other_static'], marker='*', linestyle='-', color='orange', label='Other Species (static temperature run2)')
ax2.tick_params(axis='y', labelcolor='black')
ax2.yaxis.label.set_size(14)


lines = line1 + line2 + line3 + line4
labels = [line.get_label() for line in lines]
ax1.legend(lines, labels, loc='upper right')


plt.title('Aerosol Mass Concentration')
fig.tight_layout()


plt.show()

In [ ]:

fig, ax1 = plt.subplots(figsize=(10, 6))


color = 'green'
ax1.set_xlabel('Hours past')
ax1.set_ylabel('Mixing State Index χₒ', color='black')
line1 = ax1.plot(df.index, df['OIndex'], marker='o', linestyle='-', color=color, label='Mixing state index χₒ (static temperature run1)')
line2 = ax1.plot(df.index, df['OIndex_static'], marker='x', linestyle='-', color='r', label='Mixing state index χₒ (static temperature run2)')
ax1.tick_params(axis='y', labelcolor='black')


ax2 = ax1.twinx()


x_locator = MultipleLocator(base=3)
ax1.xaxis.set_major_locator(x_locator)


color = 'blue'
ax2.set_ylabel('Temperature (K)', color=color)
line3 = ax2.plot(df.index, df['Temp'], linestyle='-', color=color, label='Temperature')
ax2.tick_params(axis='y', labelcolor=color)


lines = line1 + line2 + line3
labels = [line.get_label() for line in lines]
ax1.legend(lines, labels, loc='upper right')


plt.title('Evolution of Optical Property Mixing State Index χₒ')
fig.tight_layout()


plt.show()


In [ ]:

fig, ax1 = plt.subplots(figsize=(10, 6))


color = 'green'
ax1.set_xlabel('Hours past')
ax1.set_ylabel('Mixing State Index χ', color='black')
line1 = ax1.plot(df.index, df['gIndex'], marker='o', linestyle='-', color=color, label='Mixing state index χ (static temperature run1)')
line2 = ax1.plot(df.index, df['gIndex_static'], marker='x', linestyle='-', color='r', label='Mixing state index χ (static temperature run2)')
ax1.tick_params(axis='y', labelcolor=color)


ax2 = ax1.twinx()


x_locator = MultipleLocator(base=3)
ax1.xaxis.set_major_locator(x_locator)



color = 'blue'
ax2.set_ylabel('Temperature (K)', color=color)
line3 = ax2.plot(df.index, df['Temp'], linestyle='-', color=color, label='Temperature')
ax2.tick_params(axis='y', labelcolor=color)


lines = line1 + line2 + line3
labels = [line.get_label() for line in lines]
ax1.legend(lines, labels, loc='upper right')


plt.title('Evolution of General Mixing State Index χ')
fig.tight_layout()


plt.show()

## Static vs static comparison (Random variation analysis)

In [ ]:
InfoStatic = []

for scenario_index in range(100):
    scenario_str = f"scenario_{scenario_index:04d}"

    columns = ['Temp','Temp_static', 'DryMass_BC', 'DryMass_BC_static','DryMass_Other', 'DryMass_Other_static', 'gIndex', 'gIndex_static', 'OIndex', 'OIndex_static']
    df = pd.DataFrame(columns=columns)

    for file_index in range(2, 26):
        p = f"static2/static_scenario/{scenario_str}/out_init/urban_plume_0001_{file_index:08d}.nc"
        pmc = pmcpy.load_pmc(p)
        data = xr.open_dataset(p)

        

        Temp = float(data.temperature.values)
        DryMass_BC = pmc.get_aero_mass_conc(group[1]).sum()
        DryMass_Other = pmc.get_aero_mass_conc(group[0]).sum()
        oIndex = pmc.get_mixing_state_index(group_list=group, diversity=False)
        Index = pmc.get_mixing_state_index(drop_list=["H2O"], diversity=False)

        p = f"static/static_scenario/{scenario_str}/out_init/urban_plume_0001_{file_index:08d}.nc"
        pmc = pmcpy.load_pmc(p)
        data = xr.open_dataset(p)

        Temp_static = float(data.temperature.values)
        DryMass_BC_static = pmc.get_aero_mass_conc(group[1]).sum()
        DryMass_Other_static = pmc.get_aero_mass_conc(group[0]).sum()
        oIndex_static = pmc.get_mixing_state_index(group_list=group, diversity=False)
        Index_static = pmc.get_mixing_state_index(drop_list=["H2O"], diversity=False)

        new_row = [Temp, Temp_static, DryMass_BC, DryMass_BC_static, DryMass_Other, DryMass_Other_static, Index, Index_static, oIndex, oIndex_static]
        new_df = pd.DataFrame([new_row], columns=df.columns)
        df = pd.concat([df, new_df], ignore_index=True)

    df['Temp_diff'] = ( df['Temp'] - df['Temp_static'])
    df['OIndex_diff'] = ( df['OIndex_static'] - df['OIndex'])
    df['gIndex_diff'] = ( df['gIndex_static'] - df['gIndex'])

    average_tempdiff = np.mean(df['Temp_diff'])
    average_Odiff = np.mean(df['OIndex_diff'])
    average_gdiff = np.mean(df['gIndex_diff'])

    longitude = float(data.longitude.values)
    latitude = float(data.latitude.values)


    print(f"Done {scenario_index}")

    InfoStatic.append([df, longitude, latitude])

In [ ]:
errors = np.array([])

for i in range(100):
    df = copy.deepcopy(Info[i][0])
    df.index = df.index + 1
    list = df['gIndex_diff'].values
    errors = np.concatenate((errors, list))


In [ ]:
errors_static = np.array([])

for i in range(100):
    df = copy.deepcopy(InfoStatic[i][0])
    df.index = df.index + 1
    list = df['gIndex_diff'].values
    errors_static = np.concatenate((errors_static, list))


In [ ]:


custom_palette = sns.color_palette(["orange"])


sns.set_palette(custom_palette)


sns.set_style("whitegrid")  # Show gridlines


fig, ax = plt.subplots(figsize=(10, 3))

sns.kdeplot(data=errors_static, shade=True, label = "Random variation (static run 1- static run 2 run)", color='blue')
sns.kdeplot(data=errors, shade=True, label = "Temperature error (static run 1- time-variant run)")


plt.xlabel("Error Value", color='white')
plt.ylabel("Density", color='white')
plt.title("Optical Mixing State Index χₒ Errors vs Random Variations", color='white')
plt.legend(loc = "upper right")


fig.patch.set_facecolor('grey')  
ax.tick_params(colors='white')


# Remove the box around the plot
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['bottom'].set_visible(False)
ax.spines['left'].set_visible(False)


ax.set_facecolor('grey')  


plt.show()